In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None)

In [4]:
dftrain= pd.read_csv('dftrain.csv')
dftrain.head(1)

Unnamed: 0  Unnamed: 0.1  Pitch_Type type_of_pitch  abid Pitcher_name  \
0           0             0           1            FF     1   Jon Lester   

      Batter_name  inning  run_difference  pitcher_score  batter_score  \
0  Matt Carpenter       1               0              0             0   

   ball_count  strike_count  outs  pitch_num_ab  on_1b  on_2b  on_3b  \
0           0             0     0             1      0      0      0   

   pitcher_throws  batter_stance            weather  pitcher_id  batter_id  \
0               1              1  44 degrees, clear      452657     572761   

   top  year  
0    1  2015

In [5]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400332 entries, 0 to 1400331
Data columns (total 25 columns):
Unnamed: 0        1400332 non-null int64
Unnamed: 0.1      1400332 non-null int64
Pitch_Type        1400332 non-null int64
type_of_pitch     1400332 non-null object
abid              1400332 non-null int64
Pitcher_name      1400332 non-null object
Batter_name       1400332 non-null object
inning            1400332 non-null int64
run_difference    1400332 non-null int64
pitcher_score     1400332 non-null int64
batter_score      1400332 non-null int64
ball_count        1400332 non-null int64
strike_count      1400332 non-null int64
outs              1400332 non-null int64
pitch_num_ab      1400332 non-null int64
on_1b             1400332 non-null int64
on_2b             1400332 non-null int64
on_3b             1400332 non-null int64
pitcher_throws    1400332 non-null int64
batter_stance     1400332 non-null int64
weather           1400332 non-null object
pitcher_id        1400

### import and prep

In [6]:
#switching batters stance and pitcher throw to numerical 
#Had to fix some features: stance, pitcher's hand
#dftrain.pitcher_throws = dftrain.pitcher_throws.replace(['R', 'L'], [0,1])
#dftrain.batter_stance = dftrain.batter_stance.replace(['R', 'L'], [0,1])

In [7]:
dfall= dftrain[['Pitch_Type', 'inning', 'ball_count', 'strike_count', 'on_1b', 'on_2b', 'on_3b', 'run_difference', 'batter_stance', 'pitcher_throws']]

In [9]:
dfall.to_csv('dfall.csv')

In [40]:
#Split Data
X= dfall[['ball_count', 'inning', 'strike_count', 'on_1b', 'on_2b', 'on_3b', 'run_difference', 'batter_stance', 'pitcher_throws']].values
y=dfall['Pitch_Type'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 123)



In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

### Creating Network

In [42]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [50]:
classifier = Sequential()

In [51]:
classifier.add(Dense(6, kernel_initializer = "uniform",activation = "relu", input_dim=9))
classifier.add(Dense(7, kernel_initializer = "uniform",activation = "relu", input_dim=9))
classifier.add(Dense(8, kernel_initializer = "uniform",activation = "relu", input_dim=9))


In [52]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [53]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

### Model

In [54]:
#Training the model
classifier.fit(X_train, y_train, batch_size = 512, epochs = 10)

Epoch 1/10
1120265/1120265 [==============================] - 12s 10us/step - loss: -12.0707 - acc: 0.5292
Epoch 2/10
1120265/1120265 [==============================] - 11s 10us/step - loss: -13.0895 - acc: 0.5295
Epoch 3/10
1120265/1120265 [==============================] - 11s 10us/step - loss: -13.0895 - acc: 0.5295
Epoch 4/10
1120265/1120265 [==============================] - 12s 11us/step - loss: -13.0895 - acc: 0.5295
Epoch 5/10
1120265/1120265 [==============================] - 11s 10us/step - loss: -13.0895 - acc: 0.5295
Epoch 6/10
1120265/1120265 [==============================] - 10s 9us/step - loss: -13.0895 - acc: 0.5295: 0s - loss: -13.0855 - a
Epoch 7/10
1120265/1120265 [==============================] - 10s 9us/step - loss: -13.0895 - acc: 0.5295
Epoch 8/10
1120265/1120265 [==============================] - 11s 10us/step - loss: -13.0895 - acc: 0.5295
Epoch 9/10
1120265/1120265 [==============================] - 11s 10us/step - loss: -13.0895 - acc: 0.5295
Epoch 10/10
11

In [57]:
#Running predictions on Test Set
y_pred = classifier.predict(X_test)
print(y_pred)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [56]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[148223,      0,      0],
       [ 33776,      0,      0],
       [ 98068,      0,      0]])

In [58]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [59]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [60]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=1000, nb_epoch=10)

In [62]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)

NameError: name 'accuracies' is not defined